In [1]:
!pip -q install clearml

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50,preprocess_input
from tensorflow.keras.optimizers import Adam


from clearml import Logger
# initialisation ClearML
!clearml-init
# initialisation ClearML
!clearml-init
# Initialisation d'une Task et mise en place du Logger
from clearml import Task, Logger
task = Task.init(project_name="Tumor Classification", task_name="resnet50")
logger = Logger.current_logger()


api {   # Georges BIRANGOU's workspace   web_server: https://app.clear.ml/   api_server: https://api.clear.ml   files_server: https://files.clear.ml   credentials {     "access_key" = "WBSLZ6PVY6ZPDGHSNJLKBYK8UZDEB7"     "secret_key" = "wQs8vSAsP5frMGNt6mAE7_zmQNXYNeM0EVLgM-nCRGIqpEijIeNvKwrAsRsWMxpp2g0"   } }
ClearML SDK setup process

Please create new clearml credentials through the settings page in your `clearml-server` web app (e.g. http://localhost:8080//settings/workspace-configuration) 
Or create a free account at https://app.clear.ml/settings/workspace-configuration

In settings page, press "Create new credentials", then press "Copy to clipboard".

Paste copied configuration here:
api {   # Georges BIRANGOU's workspace   web_server: https://app.clear.ml/   api_server: https://api.clear.ml   files_server: https://files.clear.ml   credentials {     "access_key" = "WBSLZ6PVY6ZPDGHSNJLKBYK8UZDEB7"     "secret_key" = "wQs8vSAsP5frMGNt6mAE7_zmQNXYNeM0EVLgM-nCRGIqpEijIeNvKwrAsRsWMxpp

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


2025-11-03 10:17:34,462 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/d2bb63252bef42d9acbbb2c612c1e82e/experiments/9275eac7e4394c7cb1dd64578167432e/output/log


In [ ]:
# monter Google Drive (les fichiers restent sur Drive, on ne les "copie" pas en RAM)
from google.colab import drive
drive.mount('/content/drive')

# chemins fournis
train_data = "/content/drive/MyDrive/dataset_brain tumor/Training"
test_data  = "/content/drive/MyDrive/dataset_brain tumor/Testing"

print("Classes (train):", os.listdir(train_data))


Mounted at /content/drive
Classes (train): ['no_tumor', 'meningioma_tumor', 'pituitary_tumor', 'glioma_tumor']


In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# Bloc 3 : paramètres
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

# ImageDataGenerator avec préprocessing spécifique à ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # important pour ResNet50
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # split train->validation
)

# test generator : on applique uniquement preprocess_input (pas d'augmentation)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# train generator (subset training)
train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)

# validation generator (subset validation) - shuffle False to keep order for confusion matrix
validation_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)

# test generator - shuffle False for stable predictions/order
test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# infos pratiques
num_classes = train_generator.num_classes
steps_per_epoch = train_generator.samples // BATCH_SIZE
print(f"Train samples: {train_generator.samples}, Val samples: {validation_generator.samples}, Test samples: {test_generator.samples}")
print("Class indices:", train_generator.class_indices)


Found 2297 images belonging to 4 classes.
Found 573 images belonging to 4 classes.
Found 394 images belonging to 4 classes.
Train samples: 2297, Val samples: 573, Test samples: 394
Class indices: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}


In [ ]:
# base ResNet50 (gelée) + head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # on n'entraîne que la tête dans un premier temps

# tête personnalisée
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
#  compilation
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,851,012 (90.98 MB)

 Trainable params: 263,044 (1.00 MB)

 Non-trainable params: 23,587,968 (89.98 MB)

In [ ]:
#callbacks
early_stop = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)


In [11]:
!pip install tensorflow==2.15.0 keras==2.15.0 --quiet
import tensorflow as tf
print(tf.__version__)


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0
2.19.0


In [12]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stop, reduce_lr]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 2009s 28s/step - accuracy: 0.4112 - loss: 1.6511 - val_accuracy: 0.6545 - val_loss: 0.8705 - learning_rate: 1.0000e-04
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 51s 710ms/step - accuracy: 0.6935 - loss: 0.8603 - val_accuracy: 0.7016 - val_loss: 0.7536 - learning_rate: 1.0000e-04
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 53s 736ms/step - accuracy: 0.7386 - loss: 0.6620 - val_accuracy: 0.7382 - val_loss: 0.7097 - learning_rate: 1.0000e-04
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 54s 746ms/step - accuracy: 0.7946 - loss: 0.5853 - val_accuracy: 0.7400 - val_loss: 0.6406 - learning_rate: 1.0000e-04
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 53s 732ms/step - accuracy: 0.8024 - loss: 0.5363 - val_accuracy: 0.7487 - val_loss: 0.6512 - learning_rate: 1.0000e-04
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 53s 744ms/step - accuracy: 0.8355 - loss: 0.4688 - val_accuracy: 0.8045 - val_loss: 0.5863 - learning_rate: 1.0000e-04
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 52s 717ms/step - acc

In [ ]:
#évaluation & reporting ClearML

# 1) Récupérer l'historique (train/val) et envoyer scalars epoch par epoch
train_loss = history.history.get("loss", [])
val_loss = history.history.get("val_loss", [])
train_acc = history.history.get("accuracy", [])
val_acc = history.history.get("val_accuracy", [])

for epoch in range(len(train_loss)):
    logger.report_scalar("loss", "train", train_loss[epoch], epoch)
    logger.report_scalar("loss", "val", val_loss[epoch], epoch)
    logger.report_scalar("accuracy", "train", train_acc[epoch], epoch)
    logger.report_scalar("accuracy", "val", val_acc[epoch], epoch)

# 2) Évaluation sur le jeu de test (et log en tant que point final)
test_loss, test_acc = model.evaluate(test_generator, verbose=0)
print(f"Test Loss: {test_loss:.4f}  Test Accuracy: {test_acc:.4f}")

# envoyer test scalars (itération = nombre d'epochs pour qu'ils apparaissent à la fin)
iteration_point = len(train_loss)
logger.report_scalar("accuracy", "test", test_acc, iteration_point)
logger.report_scalar("loss", "test", test_loss, iteration_point)

# 3) Matrice de confusion - validation
import numpy as np
y_val_true = validation_generator.classes  # labels réels
# prédictions (assure steps pour couvrir tout)
y_val_pred_proba = model.predict(validation_generator, verbose=0)
y_val_pred = np.argmax(y_val_pred_proba, axis=1)

cm_val = confusion_matrix(y_val_true, y_val_pred)
fig_val, ax = plt.subplots(figsize=(6,5))
sns.heatmap(cm_val, annot=True, fmt="d", cmap="Blues", ax=ax)
ax.set_xlabel("Prédit"); ax.set_ylabel("Réel"); ax.set_title("Confusion Matrix - Validation")

logger.report_matplotlib_figure(
    title="Confusion Matrix - Validation",
    series="evaluation",
    iteration=iteration_point,
    figure=fig_val
)
plt.close(fig_val)

# 4) Matrice de confusion - test
y_test_true = test_generator.classes
y_test_pred_proba = model.predict(test_generator, verbose=0)
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

cm_test = confusion_matrix(y_test_true, y_test_pred)
fig_test, ax = plt.subplots(figsize=(6,5))
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Greens", ax=ax)
ax.set_xlabel("Prédit"); ax.set_ylabel("Réel"); ax.set_title("Confusion Matrix - Test")

logger.report_matplotlib_figure(
    title="Confusion Matrix - Test",
    series="evaluation",
    iteration=iteration_point,
    figure=fig_test
)
plt.close(fig_test)

# 5) classification reports printed locally
from sklearn.metrics import classification_report
print("Validation classification report:")
print(classification_report(y_val_true, y_val_pred, target_names=list(train_generator.class_indices.keys())))
print("Test classification report:")
print(classification_report(y_test_true, y_test_pred, target_names=list(train_generator.class_indices.keys())))


Test Loss: 1.2345  Test Accuracy: 0.7081
Validation classification report:
                  precision    recall  f1-score   support

    glioma_tumor       0.89      0.85      0.87       165
meningioma_tumor       0.75      0.76      0.75       164
        no_tumor       0.91      0.89      0.90        79
 pituitary_tumor       0.84      0.87      0.85       165

        accuracy                           0.83       573
       macro avg       0.84      0.84      0.84       573
    weighted avg       0.84      0.83      0.83       573

Test classification report:
                  precision    recall  f1-score   support

    glioma_tumor       0.85      0.29      0.43       100
meningioma_tumor       0.60      0.86      0.71       115
        no_tumor       0.71      0.90      0.79       105
 pituitary_tumor       0.90      0.77      0.83        74

        accuracy                           0.71       394
       macro avg       0.77      0.70      0.69       394
    weighted avg      